In [1]:
import re
import pandas as pd
import numpy as np

In [4]:
issues_df = pd.read_csv('GH_issues_clean_labeled.csv')
issues_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241934 entries, 0 to 241933
Data columns (total 19 columns):
repo_name                241934 non-null object
event_type               241934 non-null object
actor_id                 241934 non-null int64
actor_name               241934 non-null object
issue_status             241934 non-null object
issue_api_url            241934 non-null object
issue_url                241934 non-null object
issue_creator_name       241934 non-null object
issue_creator_api_url    241934 non-null object
issue_creator_url        238954 non-null object
comment_count            241934 non-null float64
issue_id                 241934 non-null float64
issue_number             241934 non-null float64
issue_created_at         241934 non-null object
issue_updated_at         241934 non-null object
issue_closed_at          190189 non-null object
issue_title              241933 non-null object
issue_body               234972 non-null object
class_label            

In [10]:
issues_df['issue_title'] = issues_df.issue_title.fillna('')
issues_df['issue_body'] = issues_df.issue_body.fillna('')
issues_df['description'] = issues_df['issue_title'].map(str) + ' ' + issues_df['issue_body']

In [88]:
issues_df.class_label.value_counts()

0    241685
2       249
Name: class_label, dtype: int64

In [89]:
prs_df = pd.read_csv('GH_prs_clean_labeled.csv')
prs_df.info()

/home/dsarkar/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359804 entries, 0 to 359803
Data columns (total 28 columns):
repo_name                   359804 non-null object
event_type                  359804 non-null object
actor_id                    359804 non-null int64
actor_name                  359804 non-null object
pr_status                   359804 non-null object
pr_id                       359804 non-null int64
pr_number                   359804 non-null int64
pr_api_url                  359804 non-null object
pr_url                      359804 non-null object
pr_diff_url                 359804 non-null object
pr_patch_url                359804 non-null object
pr_creator_name             355496 non-null object
pr_creator_api_url          355496 non-null object
pr_creator_url              350254 non-null object
pr_created_at               355497 non-null object
pr_updated_at               355497 non-null object
pr_closed_at                329332 non-null object
pr_merged_at             

In [90]:
prs_df['pr_title'] = prs_df.pr_title.fillna('')
prs_df['pr_body'] = prs_df.pr_body.fillna('')
prs_df['description'] = prs_df['pr_title'].map(str) + ' ' + prs_df['pr_body']

In [91]:
prs_df.class_label.value_counts()

0    359475
2       329
Name: class_label, dtype: int64

In [92]:
df = pd.concat([issues_df[['description', 'class_label']], 
                prs_df[['description', 'class_label']]], axis=0).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601738 entries, 0 to 601737
Data columns (total 2 columns):
description    601738 non-null object
class_label    601738 non-null int64
dtypes: int64(1), object(1)
memory usage: 9.2+ MB


In [93]:
df.head()

,description,class_label
0,add a filer aware clone of cms.plugins.flash,0
1,DocumentFragment#xpath fails to find specific ...,0
2,Unable to output the HTML::Document in the ori...,0
3,Javadoc Complete thorough Javadoc of all publi...,0
4,HTTP proxy tunneling,0


In [94]:
df.class_label.value_counts()

0    601160
2       578
Name: class_label, dtype: int64

In [95]:
def security_vulnerability_labeler(issue_description):
    strong_vuln_pattern = ('(?i)(advisory|attack|(un)?authoriz(e|ation)|'
                           'clickjack|cross.site|csrf|\\bCVE.*?\\b|deadlock|'
                           'denial.of.service|\\bEOP\\b|exploit|hijack|'
                           'infinite.loop|malicious|\\bNVD\\b|OSVDB|'
                           '\\bRCE\\b|\\bReDoS\\b|\\bDDoS\\b|remote.code.execution|'
                           'security|victim|\\bvuln|\\bXEE\\b|\\bXSRF\\b|'
                           '\\bXSS\\b|\\bXXE\\b)')

    medium_vuln_pattern = ('(?i)(authenticat(e|ion)|brute force|bypass|'
                           'constant.time|crack|credential|\\bDoS\\b|'
                           'expos(e|ing)|hack|harden|injection|lockout|'
                           'overflow|password|\\bPoC\\b|proof.of.concept|'
                           'poison|priv(ilege|elege|elage|lage)|\\b(in)?secur(e|ity)|'
                           '(de)?serializ|spoof|timing|traversal)')

    low_vuln_pattern = ('(?i)(abuse|compliant|constant.time|credential|\\bcrypto|'
                        'escalate|exhaustion|forced|infinite|RFC\\d{4,5})')
    
    if (re.findall(low_vuln_pattern, issue_description, re.I) or
        re.findall(medium_vuln_pattern, issue_description, re.I) or
        re.findall(strong_vuln_pattern, issue_description, re.I)):
        return 1
    else:
        return 0

In [97]:
negative_df = df[df.class_label == 0]
positive_df = df[df.class_label == 2]

In [104]:
negative_df['label'] = negative_df.apply(lambda row: security_vulnerability_labeler(row.description), axis=1)

/home/dsarkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [109]:
positive_df['label'] = positive_df['class_label']

/home/dsarkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [111]:
df = pd.concat([positive_df[['description', 'label']],
                negative_df[['description', 'label']]], axis=0).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601738 entries, 0 to 601737
Data columns (total 2 columns):
description    601738 non-null object
label          601738 non-null int64
dtypes: int64(1), object(1)
memory usage: 9.2+ MB


In [112]:
df['label'].value_counts()

0    517487
1     83673
2       578
Name: label, dtype: int64

In [113]:
df.to_csv('GH_processed_labeled_issues_prs.csv', index=False)